In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
# df_artists = pd.read_csv(r'/home/agastya/Documents/Python/Praxis/ML/Personal Practice/Spotify_Project/data/data_by_artist.csv', ',')

In [4]:
df_genres = pd.read_csv(r'/content/drive/My Drive/Spotify Project/data_by_genres.csv', ',')

In [5]:
df_w_genres = pd.read_csv(r'/content/drive/My Drive/Spotify Project/data_w_genres.csv', ',')

# Convert genres from string to list 
import re
pattern = re.compile(r'\w+\s*?\w+', re.IGNORECASE)
df_w_genres['genres'] = df_w_genres['genres'].map(lambda x: re.findall(pattern, x))

# Drop Unnamed column
df_w_genres.drop('Unnamed: 0', axis=1, inplace=True)

# Change fields of no genres to None
df_w_genres['genres'] = df_w_genres['genres'].map(lambda x: np.nan if len(x) == 0 else x)


In [6]:
# Remove all songs with speechiness over 0.8
df_w_genres = df_w_genres[df_w_genres['speechiness']<0.8]

In [7]:
# Separate artists with only music (no vocals) to a separate dataset. Drop these from the main set
non_vocal_artists = df_w_genres[df_w_genres['instrumentalness']>0.5]
vocal_artists = df_w_genres[df_w_genres['instrumentalness']<0.51]

In [14]:
# Split into artists with genres and artists without genres (these will be the test and train sets)
with_genre_vocal_artists = vocal_artists[~vocal_artists['genres'].isna()]
no_genre_vocal_artists = vocal_artists[vocal_artists['genres'].isna()]

In [15]:
# Split into artists with genres and artists without genres (these will be the test and train sets)
with_genre_non_vocal_artists = non_vocal_artists[~non_vocal_artists['genres'].isna()]
no_genre_non_vocal_artists = non_vocal_artists[non_vocal_artists['genres'].isna()]

In [16]:
with_genre_non_vocal_artists.shape, no_genre_non_vocal_artists.shape 

((2572, 16), (1875, 16))

In [17]:
with_genre_vocal_artists.shape, no_genre_vocal_artists.shape

((15440, 16), (7583, 16))

In [18]:
vocal_artists['artists'].name

'artists'

In [19]:
def standardize_tempo(x):
  if x.name != 'tempo':
    return x
  else:
    min_tempo = x.min()
    max_tempo = x.max()
    return (x-min_tempo)/(max_tempo-min_tempo)

# Distance Calculation #
### Still need to change the dataframe names inside this ###


In [ ]:
# Try 1 - [acousticness, danceability, instrumentalness, tempo, valence]

try_1_genres = df_genres.loc[:, ['acousticness', 'danceability', 'instrumentalness', 'tempo', 'valence']]
try_1_genres['tempo'] = try_1_genres.apply(standardize_tempo)

try_1_artists = no_genre_artists.loc[:, ['acousticness', 'danceability', 'instrumentalness', 'tempo', 'valence']]
try_1_artists['tempo'] = try_1_artists.apply(standardize_tempo)

try_1_artists = np.array(try_1_artists)
try_1_genres = np.array(try_1_genres) 

NameError: ignored

In [ ]:
artist_genre_distance_try_1 = []
artist_genre_indexes_try_1 = []

for artist_feature in try_1_artists:
    distance_vals = []
    for genre_feature in try_1_genres:
        distance_vals.append(np.linalg.norm(artist_feature-genre_feature))
    # Take only top 10
    distance_vals = np.array(distance_vals)
    
    artist_genre_distance_try_1.append(distance_vals[distance_vals.argsort()][:10])
    artist_genre_indexes_try_1.append(distance_vals.argsort()[:10])

In [ ]:
no_genre_artists.iloc[0]

artists             Orchestra Studio 7
acousticness                  0.342315
danceability                  0.561378
duration_ms                     203098
energy                        0.392441
instrumentalness              0.467496
liveness                       0.19049
loudness                      -14.3325
speechiness                  0.0542639
tempo                          116.684
valence                       0.532022
popularity                   0.0210084
key                                  5
mode                                 1
count                              238
genres                             NaN
Name: 72, dtype: object

In [ ]:
df_genres.iloc[artist_genre_indexes_try_1[0]]

,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode
2057,rhode island indie,0.248761,0.505667,188386.166667,0.728417,0.439298,0.148358,-8.770500,0.058208,132.339750,0.581667,41.083333,4,1
331,british industrial,0.260124,0.596156,253703.606803,0.632908,0.501976,0.157053,-13.414348,0.061914,121.039166,0.560386,29.080612,0,1
231,belgian indie,0.467000,0.564250,286366.500000,0.688750,0.486250,0.100200,-8.171500,0.036225,129.002000,0.506000,46.500000,10,1
259,bleep techno,0.243382,0.572361,273238.888889,0.736333,0.507456,0.208069,-8.836778,0.069403,130.764778,0.614436,43.555556,11,1
1605,modern dream pop,0.408820,0.535250,250775.749036,0.596949,0.386814,0.137330,-8.953715,0.037197,124.788873,0.462639,49.222039,2,1
1240,instrumental soul,0.381695,0.602435,252408.701452,0.539435,0.518661,0.135659,-11.004841,0.047014,115.386640,0.607844,32.983241,10,1
1865,palm desert scene,0.203318,0.482522,232229.779894,0.737249,0.427242,0.192502,-8.504152,0.056501,126.326448,0.528291,44.356878,9,1
797,downtempo,0.266034,0.603996,297572.058136,0.579686,0.424080,0.166288,-9.493431,0.066456,118.680109,0.429592,49.203524,7,1
2206,smooth jazz,0.337245,0.632402,293887.727558,0.539757,0.417892,0.139156,-10.970289,0.040750,112.243773,0.583466,37.752913,5,1
123,atmospheric post-rock,0.252210,0.668000,331466.500000,0.773500,0.478055,0.144000,-5.724000,0.056000,119.999500,0.483000,23.500000,11,1


In [ ]:
# Try 2 - [acousticness, danceability, instrumentalness, tempo,energy, valence, energy]

try_2_genres = df_genres.loc[:, ['acousticness', 'danceability', 'instrumentalness', 'energy', 'tempo', 'valence']]
try_2_genres['tempo'] = try_2_genres['tempo'].map(standardize_tempo_for_genre)

try_2_artists = no_genre_artists.loc[:, ['acousticness', 'danceability', 'instrumentalness', 'energy', 'tempo', 'valence']]
try_2_artists['tempo'] = try_2_artists['tempo'].map(standardize_tempo_for_artist)

try_2_artists = np.array(try_2_artists)
try_2_genres = np.array(try_2_genres) 

In [ ]:
artist_genre_distance_try_2 = []
artist_genre_indexes_try_2 = []

for artist_feature in try_2_artists:
    distance_vals = []
    for genre_feature in try_2_genres:
        distance_vals.append(np.linalg.norm(artist_feature-genre_feature))
    # Take only top 10
    distance_vals = np.array(distance_vals)
    
    artist_genre_distance_try_2.append(distance_vals[distance_vals.argsort()][:10])
    artist_genre_indexes_try_2.append(distance_vals.argsort()[:10])

In [ ]:
df_genres.iloc[artist_genre_indexes_try_1[4]]

,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode
2039,rebetiko,0.959770,0.533861,193814.327377,0.376418,0.336413,0.193219,-11.768717,0.053218,114.612810,0.716307,2.008617,7,1
2535,vintage old-time,0.968875,0.578806,184259.888889,0.267389,0.304343,0.192260,-12.567111,0.069199,122.886611,0.737694,4.125000,10,1
1247,irish banjo,0.837333,0.483000,172618.000000,0.601333,0.248893,0.693333,-11.168333,0.167533,128.974333,0.773667,25.666667,2,1
2541,vintage tango,0.945190,0.564508,177501.059780,0.310706,0.356746,0.243507,-11.680452,0.101448,116.664835,0.643147,4.227987,2,1
1844,orquesta tipica,0.924137,0.607385,163619.052656,0.321117,0.404575,0.291078,-11.650894,0.122958,133.062939,0.700014,4.205128,9,1
418,carnatic vocal,0.907121,0.550103,318522.668240,0.388231,0.253515,0.214114,-11.422840,0.091707,113.549147,0.680970,9.106392,10,1
1079,greek clarinet,0.844585,0.476566,171165.840931,0.522081,0.293887,0.231821,-9.599911,0.067806,110.036666,0.723124,0.000000,2,1
2534,vintage jazz,0.928986,0.610030,183929.619837,0.317587,0.329331,0.225722,-12.599809,0.076543,120.875787,0.654530,6.689927,5,1
1084,greek swing,0.978595,0.571524,191273.166667,0.336893,0.254362,0.230271,-13.069369,0.051024,113.645512,0.683095,0.595238,7,1
517,classic bollywood,0.885505,0.502630,230690.737137,0.354426,0.308211,0.213069,-11.773005,0.097147,109.769526,0.633942,8.191785,7,1


In [ ]:
df_w_genres.head(1)

,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,genres
0,Francisco Canaro,0.983072,0.654711,177776.513454,0.292622,0.490675,0.201118,-11.733373,0.111007,123.608786,0.746469,0.054257,2,1,2267,"[tango, vintage tango]"


In [ ]:
df_w_genres[~df_w_genres['genres'].isna()].shape

(18056, 16)

# Multilabel Classification #
**Vocal Artists Only**

In [31]:
# Set vocal genres
vocal_genres = df_genres[df_genres['instrumentalness']<0.51]

In [34]:
vocal_genres = vocal_genres.loc[:, ['genres','acousticness', 'danceability', 'instrumentalness', 'energy', 'tempo', 'valence']]

In [36]:
vocal_genres = vocal_genres.apply(standardize_tempo)

In [62]:
vocal_training_df = with_genre_vocal_artists.loc[:, ['genres', 'acousticness', 'danceability', 'instrumentalness', 'energy', 'tempo', 'valence']]
vocal_training_df = vocal_training_df.apply(standardize_tempo)

In [63]:
# Converting the list to a string, so that the str.get_dummies can be used
vocal_training_df['genres'] = vocal_training_df['genres'].map(lambda x: ','.join(x))

In [64]:
vocal_training_df.head(1)

,genres,acousticness,danceability,instrumentalness,energy,tempo,valence
0,"tango,vintage tango",0.983072,0.654711,0.490675,0.292622,0.567682,0.746469


In [65]:
vocal_genres.head(1)

,genres,acousticness,danceability,instrumentalness,energy,tempo,valence
1,432hz,0.48507,0.33,0.480393,0.463084,0.434946,0.217675


In [73]:
# Merge the two sets - vocal training df and vocal genres
vocal_training_df = vocal_genres.append(vocal_training_df)

In [77]:
vocal_genre_dummies = vocal_training_df['genres'].str.get_dummies(',')

In [83]:
vocal_training_df.drop('genres', inplace=True, axis=1)

In [74]:
vocal_testing_df = no_genre_vocal_artists.loc[:, ['acousticness', 'danceability', 'instrumentalness', 'energy', 'tempo', 'valence']]
vocal_testing_df = vocal_testing_df.apply(standardize_tempo)

In [ ]:
# Delete the old variables
del [df_genres, df_w_genres, no_genre_non_vocal_artists, no_genre_vocal_artists, non_vocal_artists, vocal_artists, with_genre_non_vocal_artists, with_genre_vocal_artists]

In [79]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [84]:
dt = DecisionTreeClassifier()
dt.fit(vocal_training_df, vocal_genre_dummies)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
adb = MultiOutputClassifier(DecisionTreeClassifier())
adb.fit(vocal_training_df, vocal_genre_dummies)

In [ ]:
vocal_genre_labels = adb.predict(vocal_testing_df)

In [ ]:
rfc = RandomForestClassifier(30, n_jobs=-1)
rfc.fit(vocal_training_df, vocal_genre_dummies)

In [95]:
knn = KNeighborsClassifier(3, 'distance')
knn.fit(vocal_training_df, vocal_genre_dummies)
vocal_genre_labels = knn.predict(vocal_testing_df)

In [96]:
vocal_genre_labels = knn.predict(vocal_testing_df)

In [97]:
vocal_genre_labels.sum(axis=0).mean() # On average, 8 genres for each artist!!

1.8491263440860215

In [89]:
vocal_genre_labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [87]:
(vocal_genre_labels.sum(axis=0)==0).sum() # 1266 without a genre

1285

In [98]:
vocal_genre_labels = pd.DataFrame(vocal_genre_labels, columns=vocal_genre_dummies.columns)

In [99]:
# Convert the dummy results to a list of genres 
vocal_stacked_labels = vocal_genre_labels.stack().reset_index()
vocal_labels = vocal_stacked_labels[vocal_stacked_labels[0]==1].groupby('level_0').agg(lambda x:[i for i in x])['level_1']

In [100]:
no_genre_vocal_artists['predicted_genres'] = vocal_labels

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [101]:
no_genre_vocal_artists['predicted_genres'].isna().sum()

7200

In [ ]:
df_w_genres.iloc[452]

artists                R. Kelly
acousticness           0.263177
danceability           0.710147
duration_ms              281510
energy                 0.515973
instrumentalness    0.000322452
liveness               0.129537
loudness               -7.50453
speechiness             0.08106
tempo                   110.116
valence                0.551371
popularity              42.9333
key                           1
mode                          0
count                        75
genres                      NaN
Name: 452, dtype: object

In [ ]:
test_set.head(30)

,acousticness,danceability,instrumentalness,energy,tempo,valence,genres
72,0.342315,0.561378,4.674964e-01,0.392441,116.684315,0.532022,NaN
88,0.691179,0.681108,5.389575e-06,0.208925,106.152203,0.460193,NaN
90,0.812928,0.462446,3.777778e-01,0.344851,110.679294,0.474366,NaN
124,0.968614,0.379123,4.624096e-02,0.193932,103.132047,0.212032,[classical]
130,0.953272,0.459373,3.318291e-01,0.448793,111.843538,0.725680,NaN
184,0.978504,0.386752,8.133488e-01,0.145130,99.811662,0.240863,[urban contemporary]
196,0.391992,0.681281,1.276523e-05,0.333461,106.456477,0.523214,[vapor soul]
208,0.968500,0.386770,6.734909e-02,0.184687,99.984683,0.224529,NaN
228,0.912317,0.431842,2.286588e-02,0.159334,114.793575,0.334843,NaN
262,0.214349,0.675266,5.639174e-06,0.338890,101.747486,0.677376,"[doo, wop]"
